In [ ]:
import pandas as pd
# import os
# from datetime import datetime

# 配置路径
date_str = "2026-01-09"
path = f"./data/{date_str}/"

# 1. 加载数据
def load_jsl_data(p, d):
    # 基础数据、下修、回售、强赎
    df = pd.read_csv(f"{p}jisilu_cb_data_{d}.csv")
    df_adj = pd.read_csv(f"{p}jisilu_cb_adjust_{d}.csv")
    df_put = pd.read_csv(f"{p}jisilu_cb_put_{d}.csv")
    df_red = pd.read_csv(f"{p}jisilu_cb_redeem_{d}.csv")
    
    # 统一转换代码列为字符串，防止关联失败
    for d_frame in [df, df_adj, df_put, df_red]:
        d_frame['代码'] = d_frame['代码'].astype(str)

    # 以基础表为主，左关联
    res = df.merge(df_adj[['代码', '下修提示', '下修次数']], on='代码', how='left') \
            .merge(df_put[['代码', '回售收益', '回售起始日期']], on='代码', how='left') \
            .merge(df_red[['代码', '强赎天数', '强赎状态']], on='代码', how='left')
    return res

df_all = load_jsl_data(path, date_str)


def get_top_20_pie(df):
    # --- 1. 硬性风险过滤 ---
    # 剔除强赎已满足或即将满足的（避开那 1% 的博弈）
    df = df[df['强赎天数'].fillna(0).astype(int) < 12]
    # 剔除临期且溢价标的 (剩余年限 < 0.6)
    df = df[df['剩余年限'] > 0.6]
    # 剔除规模过大的（> 15亿），确保弹性
    df = df[df['剩余规模'] < 15]
    
    # --- 2. 计算博弈评分 ---
    # 因子 A：剩余年限在 2 年附近的加分 (越接近 2 分越高)
    df['time_score'] = df['剩余年限'].apply(lambda x: 100 - abs(x - 2.0) * 20)
    
    # 因子 B：规模因子（小市值加分）
    df['size_score'] = (15 - df['剩余规模']) * 5
    
    # 因子 C：大股东未减持加分 (假设列名为 '股东配售率')
    # 这里我们认为配售率高且现价不高的标的，股东更有拉升欲望
    df['holder_score'] = df['股东配售率'].fillna(0) * 0.5
    
    # 综合得分
    df['total_score'] = df['time_score'] + df['size_score'] + df['holder_score']
    
    # --- 3. 排序并取前 20 ---
    # 限制现价在 130 以下，寻找相对低位
    top_20 = df[df['现价'] < 130].sort_values('total_score', ascending=False).head(20)
    return top_20[['代码', '转债名称', '现价', '转股溢价率', '剩余规模', '剩余年限', 'total_score']]

top_pie = get_top_20_pie(df_all)